### From Load and Reprocessing

In [1]:
import pandas as pd
import plotly.express as px

from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [ ]:
%run 'Load_and_Preprocessing.ipynb'

### Merge dataframes for analytics
df_orders: orders with tip <br>
df_order_products: ordered products with informations of the order, department and aisle.

In [3]:
df_order_products = df_order_products
df_orders = df_orders

In [4]:
df_orders['tip'] = df_orders['tip']*1
#df_tip_to_merge = df_orders[['user_id', 'order_number', 'order_hour_of_day', 'days_since_prior_order', 'tip']]

In [5]:
df_orders = df_orders.sort_values(['user_id', 'order_number'])

In [6]:
df_orders['tipped_before'] = df_orders.groupby('user_id')['tip'].shift(1)
df_orders['tipped_before_last'] = df_orders.groupby('user_id')['tip'].shift(2)

In [ ]:
df_orders

In [ ]:
df_orders_fig = df_orders[df_orders['tipped_before'].notna()]
path=['tipped_before', 'tip']
px.parallel_categories(df_orders_fig, dimensions=path, color='tip', title='Tip if tipped before')

In [ ]:
print(f'Haben Trinkgeld bei der vorherigen Bestellung gegeben: {round(df_orders_fig['tipped_before'].mean()*100, 3)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie vorher Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][df_orders_fig['tipped_before']==1].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie vorher kein Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][df_orders_fig['tipped_before']==0].mean()*100, 1)}%')


In [ ]:
# 1.2
df_orders_fig = df_orders[df_orders['tipped_before_last'].notna()]
path=['tipped_before_last', 'tipped_before', 'tip']
px.parallel_categories(df_orders_fig, dimensions=path, color='tip', title='Tip if tipped before')

In [ ]:
print(f'Haben Trinkgeld bei der vor-vorherigen Bestellung gegeben: {round(df_orders_fig['tipped_before_last'].mean()*100, 3)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vor-vorherigen Bestellung Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][df_orders_fig['tipped_before_last']==1].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vor-vorherigen Bestellung kein Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][df_orders_fig['tipped_before_last']==0].mean()*100, 1)}%')


In [ ]:
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei den beiden voherigen Bestellungen Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][(df_orders_fig['tipped_before_last']==1) & (df_orders_fig['tipped_before']==1)].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei den beiden voherigen Bestellungen kein Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][(df_orders_fig['tipped_before_last']==0) & (df_orders_fig['tipped_before']==0)].mean()*100, 1)}%')

In [ ]:
#aus 1.1 mit reduziertem Datensatz
print(f'Haben Trinkgeld bei der vorherigen Bestellung gegeben: {round(df_orders_fig['tipped_before'].mean()*100, 3)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vorherigen Bestellung Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][df_orders_fig['tipped_before']==1].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vorherigen Bestellung kein Trinkgeld gegeben haben:\
 {round(df_orders_fig['tip'][df_orders_fig['tipped_before']==0].mean()*100, 1)}%')

In [14]:
# 1.3 chi quadrat test auf 2 Tabellen, dann addieren

In [ ]:
df_orders.head(3)

In [ ]:
table_1 = pd.crosstab(df_orders['tipped_before'], df_orders['tip'])
table_1

In [18]:
ch2_1, p_1, dof_1, expected_1 = chi2_contingency(table_1)
# ch2: value of test statistic chi-squared
# p: p-value
# dof: degrees of freedom
# expected: expected frequencies if independent

In [ ]:
print('Chi2-Wert=%.6f' % (ch2_1))
print('dof=%.6f' % (dof_1))
print('p-value=%.6f' % (p_1))

In [ ]:
table_2 = pd.crosstab(df_orders['tipped_before_last'], df_orders['tip'])
table_2

In [22]:
ch2_2, p_2, dof_2, expected_2 = chi2_contingency(table_2)
# ch2: value of test statistic chi-squared
# p: p-value
# dof: degrees of freedom
# expected: expected frequencies if independent

In [ ]:
print('Chi2-Wert=%.6f' % (ch2_2))
print('dof=%.6f' % (dof_2))
print('p-value=%.6f' % (p_1))

In [ ]:
ch2_gesamt = ch2_1+ch2_2
dof_gesamt = dof_1+dof_2
p_gesamt = 1-chi2.cdf(ch2_gesamt,dof_gesamt)
print('Chi2-Wert=%.6f' % (ch2_gesamt))
print('dof=%.6f' % (dof_gesamt))
print('p-value=%.6f' % (p_gesamt))